In [1]:
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def mix_and_prune(X, y, source, target):
    source_idx = np.where(y[:, source] == 1)[0]
    np.random.shuffle(source_idx)
    target_idx = np.where(y[:, target] == 1)[0]
    np.random.shuffle(target_idx)
    final_idx = np.concatenate([
        source_idx[:int(source_idx.shape[0]/2)],
        target_idx[:int(target_idx.shape[0]/2)]])
    all_idx = np.concatenate([source_idx, target_idx])
    diff_idx = np.setdiff1d(all_idx, final_idx)
    
    X = np.delete(X, diff_idx, 0)
    
    y[final_idx,target] = 1
    y = np.delete(y, diff_idx, 0)
    y = np.delete(y, source, 1)
    
    return X, y
    
def odd_even(y):
    res = np.zeros([y.shape[0], 2])
    for i in range(5):
        eve = 2 * i
        odd = 2 * i + 1
        res[y[:, eve] == 1, 0] = 1
        res[y[:, odd] == 1, 1] = 1
    
    return res

# Reading the MNIST data.
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
train_data =  mnist.train.images * 2.0 - 1.0
train_label = mnist.train.labels

test_data = mnist.test.images * 2.0 - 1.0
test_label = mnist.test.labels

# We now label digits ODD and EVEN only
private_train_data = np.copy(train_data)
private_train_label = odd_even(np.copy(train_label))

private_test_data = np.copy(test_data)
private_test_label = odd_even(np.copy(test_label))
# source = 5
# target = 6
# private_train_data, private_train_label = mix_and_prune(
#     np.copy(train_data), np.copy(train_label), source, target)

# private_test_data, private_test_label = mix_and_prune(
#     np.copy(test_data), np.copy(test_label), source, target)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
print('Original data:')
for dig in range(10):
    print('\t#(%d) = %d' % (dig, np.sum(test_label[:,dig] == 1)))
print('Private data:')
for dig in range(2):
    print('\t#(%d) = %d' % (dig, np.sum(private_test_label[:,dig] == 1)))
    
print(private_train_data.shape[1])
print(private_test_data.shape[1])

Original data:
	#(0) = 980
	#(1) = 1135
	#(2) = 1032
	#(3) = 1010
	#(4) = 982
	#(5) = 892
	#(6) = 958
	#(7) = 1028
	#(8) = 974
	#(9) = 1009
Private data:
	#(0) = 4926
	#(1) = 5074
784
784


In [4]:
# # Creating a Keras model to learn private data
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten
# from keras.layers import Conv2D, MaxPooling2D
# from keras.optimizers import SGD
# import tensorflow as tf

# # paramters
# filename = 'models/mnist'
# params = [32, 32, 64, 64, 200, 200]
# num_epochs = 10
# batch_size = 128
# train_temp = 1
# init = None
# x_dim = private_train_data.shape[1]
# y_dim = private_train_label.shape[1]
# img_dim = 28
# input_c_dim = 1
# output_c_dim = 1
# input_dim = x_dim
# label_dim = y_dim


# # Just using tensorflow to reshape the data!
# source = tf.placeholder(tf.float32, [None, input_dim], name = "source_image")
# images = tf.reshape( source, [-1, img_dim, img_dim, input_c_dim])

# with tf.Session() as sess:
#     private_train_data = sess.run(images, feed_dict = {source : private_train_data})
#     private_test_data = sess.run(images, {source : private_test_data})

# model = Sequential()

# print('New traininig data has shape %s' % str(private_train_data.shape))

# model.add(Conv2D(params[0], (3, 3),
#                         input_shape=[img_dim,img_dim,input_c_dim]))
# model.add(Activation('relu'))
# model.add(Conv2D(params[1], (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(params[2], (3, 3)))
# model.add(Activation('relu'))
# model.add(Conv2D(params[3], (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dense(params[4]))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(params[5]))
# model.add(Activation('relu'))
# model.add(Dense(2))

# def fn(correct, predicted):
#     return tf.nn.softmax_cross_entropy_with_logits(labels=correct,
#                                                    logits=predicted/train_temp)

# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# # model.compile(loss=fn,
# model.compile(loss= fn,
#               optimizer=sgd,
#               metrics=['accuracy'])

# model.fit(private_train_data, private_train_label,
#           batch_size=batch_size,
#           validation_data=(private_test_data, private_test_label),
#           nb_epoch=num_epochs,
#           shuffle=True)

In [6]:
# import os
# if not os.path.isdir('models'):
#     os.makedirs('models')
# model.save('models/odd_even')

In [7]:
from Dataset2 import Dataset2
loader = Dataset2(private_train_data, private_train_label)
print(np.sum(loader._label, 0))
print(loader._data.shape[0])

Data size: (55000, 784)
Label size: (55000, 2)
Targeted = False
[ 27027.  27973.]
55000
